In [1]:
!apt-get install sox ffmpeg libcairo2 libcairo2-dev
!apt-get install texlive-full 
from google.colab import drive
drive.mount('/content/drive')
%cd drive/MyDrive/Colab_Notebooks/perturbation_metric_test
!pip install anndata 
!pip install git+https://github.com/theislab/scgen.git
!pip install matplotlib
!pip install scanpy
!pip install pertpy
# need to run this block twice

Reading package lists... Done
Building dependency tree       
Reading state information... Done
libcairo2 is already the newest version (1.16.0-4ubuntu1).
libcairo2-dev is already the newest version (1.16.0-4ubuntu1).
ffmpeg is already the newest version (7:4.2.7-0ubuntu0.1).
sox is already the newest version (14.4.2+git20190427-2+deb11u2build0.20.04.1).
0 upgraded, 0 newly installed, 0 to remove and 24 not upgraded.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
texlive-full is already the newest version (2019.20200218-1).
0 upgraded, 0 newly installed, 0 to remove and 24 not upgraded.
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/Colab_Notebooks/perturbation_metric_test
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pk

In [ ]:
!pip install scvi-tools>=0.20.0

In [2]:
import anndata
import scanpy as sc
import scgen
import pandas as pd  
import numpy as np
import pertpy as pt
import matplotlib.pyplot as plt


INFO:pytorch_lightning.utilities.seed:Global seed set to 0
/usr/local/lib/python3.10/dist-packages/pytorch_lightning/utilities/warnings.py:53: LightningDeprecationWarning: pytorch_lightning.utilities.warnings.rank_zero_deprecation has been deprecated in v1.6 and will be removed in v1.8. Use the equivalent function from the pytorch_lightning.utilities.rank_zero module instead.
  new_rank_zero_deprecation(
/usr/local/lib/python3.10/dist-packages/pytorch_lightning/utilities/warnings.py:58: LightningDeprecationWarning: The `pytorch_lightning.loggers.base.rank_zero_experiment` is deprecated in v1.7 and will be removed in v1.9. Please use `pytorch_lightning.loggers.logger.rank_zero_experiment` instead.
  return new_rank_zero_deprecation(*args, **kwargs)


To use sccoda or tasccoda please install ete3 with pip install ete3

In [3]:
train = sc.read("./data/PapalexiSatija2021_eccite_RNA.h5ad")
sc.pp.normalize_total(train, target_sum=1e4)
sc.pp.log1p(train)
sc.pp.highly_variable_genes(train, n_top_genes=100, subset=True, min_mean=.1)
sc.pp.pca(train, use_highly_variable=True)
sc.pp.neighbors(train, use_rep='X_pca', n_neighbors=30, n_pcs=30)

In [ ]:
obs_key = 'perturbation'  # defines groups to test
distance = pt.tl.Distance('edistance', 'X_pca')
df = distance.pairwise(train, groupby=obs_key, verbose=True)
from seaborn import clustermap
clustermap(df, robust=True, figsize=(10,10))
plt.show()
#df is a data frame show results in e-distance not p-value 

In [ ]:

ctrl_stim=df['control']
print(ctrl_stim['control'])# ctrl, ctrl 
np.mean(ctrl_stim)# ctrl, stim 

In [4]:
obs_key = 'perturbation' 
etest = pt.tl.DistanceTest('edistance', n_perms=1000, obsm_key='X_pca', alpha=0.0015)
contrast = 'control' 
tab = etest(train, groupby=obs_key, contrast=contrast)




Output()

In [11]:
tab.to_csv("e_distance.csv",index=False)# this does save the name of rows

In [5]:
tab.columns.values

array(['distance', 'pvalue', 'significant', 'pvalue_adj',
       'significant_adj'], dtype=object)

In [19]:
tab_copy=tab.copy()
tab_copy['name']=tab.index

In [21]:
tab_copy.to_csv("e_distance.csv",index=False)

In [22]:
import pandas as pd
tab2 = pd.read_csv("e_distance.csv") 
tab2

,distance,pvalue,significant,pvalue_adj,significant_adj,name
0,0.017370,0.661,False,0.999996,False,STAT2g2
1,0.029570,0.007,False,0.325227,False,CAV1g4
2,0.315472,0.067,False,0.943171,False,STAT1g2
3,0.010651,0.016,False,0.581462,False,CD86g1
4,0.037443,0.001,True,0.094302,False,IRF7g2
...,...,...,...,...,...,...
94,0.333902,0.186,False,0.996138,False,SPI1g3
95,0.051816,0.766,False,0.999996,False,PDCD1LG2g2
96,0.336118,0.405,False,0.999913,False,CUL3g3
97,1.248222,0.001,True,0.094302,False,SPI1g2


In [14]:
tab['pvalue_adj']['control'] #1

1.0

In [15]:
sum_stim=sum(tab['pvalue_adj'])-1
sum_stim/98  #0.567399

0.5673994175166678

In [16]:
print(tab['pvalue']['control']) #1 
(sum(tab['pvalue'])-1)/98 #0.17

1.0


0.17380612244897967